# Coordinates

Container for describing the coordinates of a PODPAC datasource.
Coordinates are created from a list or dict containing BaseCoordinates1d objects (Coordinates1d or StackedCoordinates).

Constructors:

- `Coordinates()`
- `Coordinates([coords1d, coords1d])`
- `Coordinates([StackedCoordinates([coords1d, coords1d]), coords1d])`
- `Coordinates([(coords1d, coords1d), coords1d])`
- `Coordinates([array1d, array1d], dims=['lat', 'lon'])`
- `Coordinates([(array1d, array1d), array], dims=['lat_lon', 'time'])`
- `Coordinates([array2d, array1d], dims=['lat_lon', 'time'])`

Advanced Contructors:

- `Coordinates.from_xarray(xcoords)`: maps multi-dimensional xarray `DataArrayCoordinates` to podpac `Coordinates`
- `Coordinates.grid(...)`
- `Coordinates.points(...)`

In [2]:
import numpy as np
from podpac import Coordinates

### Grid Coordinates

In [38]:
# coordinates specifying a n x n grid
grid_coords = Coordinates([np.linspace(-10, 10, 21), np.linspace(-10, 10, 21)], dims=['lat', 'lon'])

In [39]:
# get tuple of dimensions 
grid_coords.dims

('lat', 'lon')

In [5]:
# get tuple of unstacked dimensions (all individual dimensions)
grid_coords.udims

('lat', 'lon')

In [6]:
# ArrayCoordinates1d within the main Coordinates container
grid_coords['lat']

ArrayCoordinates1d(lat): Bounds[-10.0, 10.0], N[21], ctype['midpoint']

In [7]:
# get array of coordinate values
grid_coords['lat'].coordinates

array([-10.,  -9.,  -8.,  -7.,  -6.,  -5.,  -4.,  -3.,  -2.,  -1.,   0.,
         1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.])

In [8]:
# get ndims and size of coordinates
print(grid_coords.ndim)
print(grid_coords.size)

2
441


### Stacked Coordinates

In [43]:
# stacked coordinates specifying n data points
stacked_coords = Coordinates([(np.linspace(-10, 10, 21), np.linspace(-10, 10, 21))], dims=['lat_lon'])

In [44]:
# get tuple of dimensions 
stacked_coords.dims

('lat_lon',)

In [45]:
# get tuple of unstacked dimensions (all individual dimensions)
stacked_coords.udims

('lat', 'lon')

In [12]:
# StackedCoordinates within main Coordinates container
stacked_coords['lat_lon']

StackedCoordinates
	lat_lon[lat]: ArrayCoordinates1d(lat): Bounds[-10.0, 10.0], N[21], ctype['midpoint']
	lat_lon[lon]: ArrayCoordinates1d(lon): Bounds[-10.0, 10.0], N[21], ctype['midpoint']

In [13]:
# multi-index array of values
stacked_coords['lat_lon'].coordinates

MultiIndex(levels=[[-10.0, -9.0, -8.0, -7.0, -6.0, -5.0, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0], [-10.0, -9.0, -8.0, -7.0, -6.0, -5.0, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]],
           labels=[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]],
           names=['lat', 'lon'])

In [14]:
# ArrayCoordinates1D within StackedCoordinates
stacked_coords['lat']

ArrayCoordinates1d(lat): Bounds[-10.0, 10.0], N[21], ctype['midpoint']

In [15]:
# Individual coordinate values within StackedCoordinates
stacked_coords['lat'].coordinates

array([-10.,  -9.,  -8.,  -7.,  -6.,  -5.,  -4.,  -3.,  -2.,  -1.,   0.,
         1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.])

### Stacked and Grid Coordinates

In [47]:
# stacked coordinates specifying n data points
hybrid_coords = Coordinates([(np.linspace(-10, 10, 21), np.linspace(-10, 10, 21)), np.linspace(0, 10, 11)], dims=['lat_lon', 'time'])

In [48]:
# get tuple of dimensions 
hybrid_coords.dims

('lat_lon', 'time')

In [49]:
# get tuple of unstacked dimensions (all individual dimensions)
hybrid_coords.udims

('lat', 'lon', 'time')

## Advanced Coordinate Types

### Dependent Coordinates

Dependent coordinates are coordinates defined at a set of N-d input indexes.

In [72]:
from podpac.core.coordinates import RotatedCoordinates

lat = np.linspace(0, 1, 12).reshape(3, 4)
lon = np.linspace(10, 20, 12).reshape(3, 4)

# coerce into DependentCoordinates
c = Coordinates([[lat, lon]], dims=['lat,lon'])
c

Coordinates
	lat,lon[lat]: DependentCoordinates(i,j->lat): Bounds[0.0, 1.0], shape(3, 4), ctype[point]
	lat,lon[lon]: DependentCoordinates(i,j->lon): Bounds[10.0, 20.0], shape(3, 4), ctype[point]

In [73]:
c['lat'].coordinates

array([[0.        , 0.09090909, 0.18181818, 0.27272727],
       [0.36363636, 0.45454545, 0.54545455, 0.63636364],
       [0.72727273, 0.81818182, 0.90909091, 1.        ]])

In [74]:
c['lon'].coordinates

array([[10.        , 10.90909091, 11.81818182, 12.72727273],
       [13.63636364, 14.54545455, 15.45454545, 16.36363636],
       [17.27272727, 18.18181818, 19.09090909, 20.        ]])

In [76]:
# indexing dimensions
c.idims

('i', 'j')

### Rotated Coordinates

Rotated Coordinates define a gridded coordinate system rotated around an axis.

In [32]:
# using upper left corner and a step/scale
rc = RotatedCoordinates(shape=(3, 4), theta=np.pi/4, ulc=[10, 20], step=[1.0, 2.0], dims=['lat', 'lon'])
c = Coordinates([rc], dims=['lat,lon'])
c

Coordinates
	lat,lon[lat]: RotatedCoordinates(i,j->lat): Bounds[10.0, 15.65685424949238], shape(3, 4), ctype[point]
	lat,lon[lon]: RotatedCoordinates(i,j->lon): Bounds[15.757359312880714, 21.414213562373096], shape(3, 4), ctype[point]

In [77]:
# indexing dimensions
c.idims

('i', 'j')

In [34]:
# using upper left corner and lower right corner
rc = RotatedCoordinates(shape=(3, 4), theta=np.pi/4, ulc=[10, 20], lrc=[15, 17], dims=['lat', 'lon'])
c = Coordinates([rc], dims=['lat,lon'])
c

Coordinates
	lat,lon[lat]: RotatedCoordinates(i,j->lat): Bounds[10.0, 14.999999999999996], shape(3, 4), ctype[point]
	lat,lon[lon]: RotatedCoordinates(i,j->lon): Bounds[16.0, 20.999999999999996], shape(3, 4), ctype[point]

In [36]:
# using upper left corner and lower right corner
rc = RotatedCoordinates(shape=(3, 4), theta=np.pi/4, ulc=[10, 20], lrc=[15, 17], dims=['lat', 'lon'])
c = Coordinates([rc], dims=['lat,lon'])

In [38]:
# to / from gdal geotransform
rc = RotatedCoordinates(shape=(3, 4), theta=np.pi/4, ulc=[10, 20], step=[1.0, 2.0], dims=['lat', 'lon'])
c = RotatedCoordinates.from_geotransform(rc.geotransform, rc.shape, dims=rc.dims)
c

RotatedCoordinates(('lat', 'lon')): ULC[10. 20.], LRC(15.65685424949238, 17.17157287525381), rad[0.7854], shape(3, 4), ctype['point']

### Polar Coordinates

> Not yet implemented

In [39]:
from podpac.coordinates import PolarCoordinates

ImportError: cannot import name 'PolarCoordinates' from 'podpac.coordinates' (/Users/marcus/computing/creare/podpac/podpac/podpac/coordinates.py)

## Coordinate Operations

### Coordinate Indexing

In [42]:
lat = [0, 1, 2, 3, 5]
lon = [10, 20, 30, 40]
time = ['2018-01-01', '2018-01-02', '2018-01-03']

c = Coordinates([lat, lon, time], dims=['lat', 'lon', 'time'])
c

Coordinates
	lat: ArrayCoordinates1d(lat): Bounds[0.0, 5.0], N[5], ctype['midpoint']
	lon: ArrayCoordinates1d(lon): Bounds[10.0, 40.0], N[4], ctype['midpoint']
	time: ArrayCoordinates1d(time): Bounds[2018-01-01, 2018-01-03], N[3], ctype['point']

In [49]:
# select subset of coordinates by index within dimension
# elements 1 and 3 of lat
# slice 0 to 2 of lon
# element 2 of time
c[[1, 3], slice(0,2), 2]

Coordinates
	lat: ArrayCoordinates1d(lat): Bounds[1.0, 3.0], N[2], ctype['midpoint']
	lon: ArrayCoordinates1d(lon): Bounds[10.0, 20.0], N[2], ctype['midpoint']
	time: ArrayCoordinates1d(time): Bounds[2018-01-03, 2018-01-03], N[1], ctype['point']

In [59]:
# works for stacked coordinates
lat_lon = (np.linspace(0, 10, 11), np.linspace(10, 20, 11))
time = ['2018-01-01', '2018-01-02', '2018-01-03']

c = Coordinates([lat_lon, time], dims=['lat_lon', 'time'])
c

Coordinates
	lat_lon[lat]: ArrayCoordinates1d(lat): Bounds[0.0, 10.0], N[11], ctype['midpoint']
	lat_lon[lon]: ArrayCoordinates1d(lon): Bounds[10.0, 20.0], N[11], ctype['midpoint']
	time: ArrayCoordinates1d(time): Bounds[2018-01-01, 2018-01-03], N[3], ctype['point']

In [69]:
# select the 1st and 3rd element from the stacked coordinates
# select the 2nd element of time
c[[1, 3], 2]

Coordinates
	lat_lon[lat]: ArrayCoordinates1d(lat): Bounds[1.0, 3.0], N[2], ctype['midpoint']
	lat_lon[lon]: ArrayCoordinates1d(lon): Bounds[11.0, 13.0], N[2], ctype['midpoint']
	time: ArrayCoordinates1d(time): Bounds[2018-01-03, 2018-01-03], N[1], ctype['point']

### Coordinate Seletion

In [70]:
lat = [0, 1, 2, 3, 5]
lon = [10, 20, 30, 40]
time = ['2018-01-01', '2018-01-02', '2018-01-03']

c = Coordinates([lat, lon, time], dims=['lat', 'lon', 'time'])
c

Coordinates
	lat: ArrayCoordinates1d(lat): Bounds[0.0, 5.0], N[5], ctype['midpoint']
	lon: ArrayCoordinates1d(lon): Bounds[10.0, 40.0], N[4], ctype['midpoint']
	time: ArrayCoordinates1d(time): Bounds[2018-01-01, 2018-01-03], N[3], ctype['point']

In [71]:
c.select({'lat': [1, 3]})

KeyError: 0

In [ ]:
c.select({'lat': [1, 3]})
c.select({'lat': [1, 3], 'lon': [15, 25]})
c.select({'alt': [1, 3]}) # no affect